In [2]:
from datetime import datetime

import polars as pl

In [ ]:
df = pl.read_csv('docs/src/data/apleStock.csv', try_parse_dates=True)
df = df.sort('Date')
df

In [ ]:
annual_mean = (
    df.groupby_dynamic('Date', every='1yr').agg(pl.col('Close').mean()))
annual_mean = (
    annual_mean.with_columns(pl.col('Date').dt.year().alias('year')))

In [5]:
df = (
    pl
    .date_range(
        start=datetime(2021, 1, 1),
        end=datetime(2021, 12, 31),
        interval='1d',
        eager=True)
    .alias('time')
    .to_frame())
df.head()

time
datetime[μs]
2021-01-01 00:00:00
2021-01-02 00:00:00
2021-01-03 00:00:00
2021-01-04 00:00:00
2021-01-05 00:00:00


In [6]:
(df
 .groupby_dynamic('time', every='1mo', period='1mo', closed='left')
 .agg([
     pl
     .col('time')
     .cumcount()
     .reverse()
     .head(3)
     .alias('days_left_in_mo'),
     ((pl.col('time') - pl.col('time').first()).last().dt.days() + 1)
     .alias('days_in_mo')])
 .explode('days_left_in_mo'))

time,days_left_in_mo,days_in_mo
datetime[μs],u32,i64
2021-01-01 00:00:00,30,31
2021-01-01 00:00:00,29,31
2021-01-01 00:00:00,28,31
2021-02-01 00:00:00,27,28
2021-02-01 00:00:00,26,28
2021-02-01 00:00:00,25,28
2021-03-01 00:00:00,30,31
2021-03-01 00:00:00,29,31
2021-03-01 00:00:00,28,31
